#  Учимся соединять датафреймы с помощью merge

Домашнее задание будет разбито на этот урок (3 задачи) и следующий урок (4 задачи). Мы будем работать с набором данных, который содержит информацию о предприятиях  общественного питания в городе Москве в части наименования, точного адреса и специализации. Набор разбит на файлы:

main.csv - основная информация о предприятиях общественного питания (скачать)

    ID - уникальный номер объекта
    Name - наименование
    Address - адрес

 

net_property.csv - является ли предприятие сетевым (скачать)

    ID - уникальный номер объекта
    IsNetObject - метка, которая указывает на то, является ли предприятие сетевым

 

seats_property.csv - количество посадочных мест (скачать)

    ID - уникальный номер объекта
    SeatsCount - количество посадочных мест
    ver_ID - номер версии записи (чем выше, тем запись более актуальная)

 

type_property.csv - информация о специализации: столовая, закусочная, кафе, иное (скачать)

    ID - уникальный номер объекта
    TypeObject - специализация (тип) объекта
    is_actual - является ли запись о специализации актуальной. 0 - не актуальная, 1 - актуальная


In [1]:
import pandas as pd
import numpy as np

In [2]:
df_main = pd.read_csv('main.csv')
df_net = pd.read_csv('net_property.csv')
df_seats = pd.read_csv('seats_property.csv')
df_type = pd.read_csv('type_property.csv')

In [87]:
df_main.head(3)


,ID,Name,Address
0,20988,ГУЗ,"город Москва, улица Казакова, дом 15"
1,24828,Кафе Халяль,"город Москва, 8-я улица Текстильщиков, дом 5, ..."
2,358126,Scarface,"Российская Федерация, город Москва, внутригоро..."


In [92]:
df_net.head(3)

,ID,IsNetObject
0,369614,нет
1,201691,нет
2,362705,нет


In [179]:
df_seats.head(3)

,ID,SeatsCount,ver_ID
0,345181,0,1
1,326754,35,2
2,151037,0,1


In [89]:
df_type.head(3)

,ID,TypeObject,is_actual
0,302619,закусочная,True
1,28573,столовая,True
2,373269,кафе,False


## Задание 1

### Определите процент объектов у которых запись о специализации отсутствует или не является актуальной. Получившийся процент округлите до целых. В ответе не указывайте знак процента.

In [121]:
df = pd.merge(df_main, df_type, on='ID', how='left')


In [122]:
df

,ID,Name,Address,TypeObject,is_actual
0,20988,ГУЗ,"город Москва, улица Казакова, дом 15",столовая,True
1,24828,Кафе Халяль,"город Москва, 8-я улица Текстильщиков, дом 5, ...",NaN,NaN
2,358126,Scarface,"Российская Федерация, город Москва, внутригоро...",кафе,True
3,373052,Кулинариум,"Российская Федерация, город Москва, внутригоро...",кафетерий,False
4,303239,One price coffee,"Российская Федерация, город Москва, внутригоро...",NaN,NaN
...,...,...,...,...,...
9995,192373,Рамен Клаб,"город Москва, Мясницкая улица, дом 16",NaN,NaN
9996,204304,Папа Джонс,"город Москва, проспект Маршала Жукова, дом 59",кафе,False
9997,301366,Up Sushi,"город Москва, город Московский, улица Москвити...",кафе,False
9998,339278,Шаурма,"город Москва, улица Орджоникидзе, дом 12, стро...",предприятие быстрого обслуживания,True


In [81]:
total = df.shape[0]
count_not_act = df[(df['is_actual'] == False) | (df['is_actual'].isna())].count()[0]

print(round(count_not_act*100/total))

67


##  Задание 2

### Подтяните к исходному списку (main.csv) последние данные о числе посадочных мест для каждого объекта. Сколько всего посадочных мест получилось?


подсказки:

1. Через левый мердж присоединяем к main.csv seats_property.csv.
2. Далее сортируем по столбцам ID и ver_ID.
3. Дропаем дубликаты (subset=['ID']) ключ keep='last'
4. Суммируем SeatsCount



In [267]:
df = pd.merge(df_main, df_seats, on='ID', how='left')
df = df.sort_values(['ID', 'ver_ID']).drop_duplicates(subset='ID', keep='last')

df['SeatsCount'].sum()

188241.0

## Задание 3

### Сформируйте итоговую выгрузку в формате csv (разделитель запятая), которая будет содержать все объекты общепита из файла main.csv. Подтяните признак сетевой/не сетевой и актуальную специализацию. Отсортируйте по специализации  в алфавитном порядке. Внутри каждой специализации сортировка должна идти по убыванию ID. Объекты у которых отсутствует специализация должны идти в начале выгрузки (и тоже по убыванию ID).

Колонки, которые должны быть в csv файле:<br>
<code>ID, Name, Address, IsNetObject, TypeObject, is_actual</code>

Подсказки:

1) Оставить только актуальную специализацию<br>
2) Смержить рестораны, сетевой/несетевой, специализацию (отфильтрованнюу на шаге один) левым мержем<br>
3) Отсортировать как указано используя параметры функции сортировки<br>

In [333]:
df_type_actual = df_type[df_type.is_actual]

In [334]:
df = df_main.merge(df_net, on='ID', how='left').merge(df_type_actual, on='ID', how='left')

In [335]:
result = df.sort_values(by=['TypeObject', 'ID'], na_position='first', ascending=[True,False]) 

In [336]:
result.head()

,ID,Name,Address,IsNetObject,TypeObject,is_actual
4912,375956,Крошка Картошка,"город Москва, Люблинская улица, дом 169, корпус 2",да,NaN,NaN
5063,375954,Кофе,"Российская Федерация, город Москва, внутригоро...",NaN,NaN,NaN
7804,375834,Донеретто,"город Москва, улица Лестева, дом 6",нет,NaN,NaN
493,375830,Шаурма,"город Москва, улица Вертолётчиков, дом 2А, кор...",NaN,NaN,NaN
5521,375822,Кафе,"Российская Федерация, город Москва, внутригоро...",NaN,NaN,NaN


In [337]:
result.tail()

,ID,Name,Address,IsNetObject,TypeObject,is_actual
1317,21282,Мед. колледж № 1,"город Москва, Чуксин тупик, дом 6",NaN,столовая,True
0,20988,ГУЗ,"город Москва, улица Казакова, дом 15",нет,столовая,True
8837,20918,ГБОУ Школа № 1251 Учебный корпус 4,"город Москва, улица Алабяна, дом 15",нет,столовая,True
1382,20854,Орандж,"город Москва, Подъёмная улица, дом 12, строение 1",нет,столовая,True
1182,19724,КП МГУ закусочная 2,"город Москва, территория Ленинские Горы, дом 1",NaN,столовая,True


In [338]:
result.to_csv('merge_task3.csv', index=False)

## Задание 4

### Разбейте все сетевые объекты общепита со специализацией кафе на 4 группы (по количеству посадочных мест):

1) 0 - 5

2) 6 - 15 

3) 16 - 25

4) 26+

### В какой группе больше всего объектов? В ответе укажите номер группы и количество объектов в этой группе через пробел. Пример ответа:

    2 26


1) Объединяем все таблицы в одну, можно через merge,  how='left'
5) Оставляем в таблице  IsNetObject=='да',  is_actual==True,  TypeObject == 'кафе'
6) Добавляем новый столбец = Используем метод cut. Нарезаем df по bin, ds, можно дать имена через labels = [, , ,]
7) Дальше через groupby все группируем по новому столбцу и  ID.  И вызываем метод для подсчета значений.


In [114]:
df_main = pd.read_csv('main.csv')
df_net = pd.read_csv('net_property.csv')
df_seats = pd.read_csv('seats_property.csv')
df_type = pd.read_csv('type_property.csv')

In [130]:
df_seats = df_seats.drop_duplicates(subset='ID', keep='last')
df = df_main.merge(df_net, on='ID', how='left').merge(df_type, on='ID', how='left').merge(df_seats, on='ID', how='left')
df = df.query("IsNetObject == 'да' & is_actual == True & TypeObject == 'кафе' & SeatsCount.notnull()")
df['group'] = pd.cut(df['SeatsCount'], labels=[1, 2, 3, 4], bins=[0, 5, 15, 25, 1000],  include_lowest=True)
result = df['group'].value_counts()
print(result.idxmax(), result.max())

4 46


##  Задание 5

### Найдите id сетевых кафе у которых отсутствует запись в (main.csv). Отсортируйте id по убыванию и загрузите их.
В вашем файле должен быть только один столбец - ID. Выгрузка в формате csv (без индекса).

1. Считал main.csv, net_property.csv, type_property.csv
2. Объеденил через merge датафреймы type_property и net_property, а затем main, через how='left'.
3. Отфильтровал по колонкам: TypeObject, is_actual и IsNetObject.
4. Взял только те строки датафрейма, где Address isnull и отсортировав по убыванию ID, сохранил столбец ID в csv, index=False. 

In [159]:
df_main = pd.read_csv('main.csv')
df_net = pd.read_csv('net_property.csv')
df_type = pd.read_csv('type_property.csv')

In [166]:
df = df_type.merge(df_net, on='ID').merge(df_main, on='ID', how='left')
result = df.query("Name.isna() & IsNetObject =='да' & TypeObject == 'кафе' & is_actual")['ID'].sort_values(ascending=False)
result.to_csv('main_no_records.csv', index=False)

## Задание 6

### Коллеги из юридического отдела прислали вам два списка - адреса и id объектов. Подтяните для них информацию:

1) признак сетевой/не сетевой. Если признака нет, то оставьте поле пустым.
2) актуальную специализацию. Если ее нет, то напишите 'отсутствует'
3) последнюю информацию о числе посадочных мест. Если мест нет, то укажите 0.

Отсортируйте объекты в порядке убывания ID. Колонки переведите на русский язык т.к. коллеги не понимают англ. язык:
Ид,Наименование,Адрес,Сетевой?,Специализация,КоличМест

Загрузите файл в формате csv (разделитель запятая, без индекса).


1. Читаем все csv.
2. Читаем excel, обратите внимание что в "Список_адресов" есть скрытая страница, поэтому указываем считываемую вкладку.
3. Оставляем в "seats_property" информацию с максимальным ver_ID, а в "type_property" с is_actual == True
4. Мерджим отдельно "Список_ID" и "Список_адресов" с "main", how='inner'. Затем через "concat" объединяем данные датафреймы в единый.
5. С полученным датафреймом объединяем "net_property", "seats_property" и "type_property", how='left'.
6. Через fillna заполняем все пропуски.
7. Переименовываем столбцы, удаляем дубликаты(drop_duplicates), сортируем(sort_values) и сохраняем в csv.

In [337]:
df_main = pd.read_csv('main.csv')
df_net = pd.read_csv('net_property.csv')
df_seats = pd.read_csv('seats_property.csv')
df_type = pd.read_csv('type_property.csv')

In [338]:
id_list = pd.read_excel("Список_ID.xlsx", sheet_name="Sheet1", skiprows=1)
address_list = pd.read_excel("Список_адресов.xlsx", sheet_name="Sheet1", names=['Address'], skiprows=1)

In [339]:
df_seats = df_seats.drop_duplicates(subset='ID', keep='last')
df_type = df_type[df_type['is_actual']]

In [340]:
id_main = df_main.merge(id_list, on='ID', how='inner')
adr_main = df_main.merge(address_list, on='Address', how='inner')

df = pd.concat([id_main, adr_main]).merge(df_net, how='left').merge(df_seats, how='left').merge(df_type, how='left')
df.sort_values('ID', ascending=False)

res = df[['ID','Name','Address', 'IsNetObject','TypeObject','SeatsCount']]
res = res.drop_duplicates(subset='ID').sort_values('ID', ascending=False)

# res['IsNetObject'] = res['IsNetObject'].fillna('')
res['TypeObject'] = res['TypeObject'].fillna('отсутствует')
res['SeatsCount'] = res['SeatsCount'].fillna(0)

res.columns=['Ид','Наименование','Адрес','Сетевой?','Специализация','КоличМест']
res.to_csv('result.csv', index=False)


In [341]:
res

,Ид,Наименование,Адрес,Сетевой?,Специализация,КоличМест
78,372752,ШАУРМА,"Российская Федерация, город Москва, внутригоро...",нет,отсутствует,0.0
66,369518,Чайхана УМИД,"город Москва, Ярцевская улица, дом 34, корпус 2",нет,отсутствует,0.0
29,367457,Stars Coffee,"город Москва, Пресненская набережная, дом 2, п...",NaN,отсутствует,0.0
11,367328,Чебурешная,"город Москва, Пресненская набережная, дом 2, п...",NaN,кафе,0.0
83,365834,Кофестанция,"Российская Федерация, город Москва, внутригоро...",нет,отсутствует,9.0
...,...,...,...,...,...,...
52,27010,Премьер,"город Москва, Ленинградский проспект, дом 31А,...",нет,ресторан,83.0
63,25007,Шоколадница,"Российская Федерация, город Москва, внутригоро...",да,отсутствует,0.0
79,24219,Кафе «Чашки»,"Российская Федерация, город Москва, внутригоро...",NaN,отсутствует,0.0
0,20821,Теремок,"город Москва, Ладожская улица, дом 5",да,отсутствует,0.0


## Задание 7

### Прямо сейчас у вашего начальника идет совещание с руководством организации. Обсуждают меры поддержки наиболее пострадавших торговых центров от закрытия фуд-кортов на период коронавирусных ограничений. Вы сидите на своем рабочем месте и смотрите один из уроков 'Основы Pandas для начинающих' на степике. Вдруг в Telegram от начальника приходит сообщение:
### "Срочно посмотри сколько всего мест у общепита по адресу просп. Вернадского 86А"



In [374]:
df_main = pd.read_csv('main.csv')
df_seats = pd.read_csv('seats_property.csv').drop_duplicates(subset='ID', keep='last')


In [375]:
df = df_main.merge(df_seats, how='left')
int(df[df.Address.str.contains('Вернадского, дом 86А')].SeatsCount.sum())

238